In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)

new_transactions = pd.read_csv('Elo_merchant_recommendation/new_merchant_transactions.csv',
                               parse_dates=['purchase_date'])

historical_transactions = pd.read_csv('Elo_merchant_recommendation/historical_transactions.csv',
                                      parse_dates=['purchase_date'])

def binarize(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

historical_transactions = binarize(historical_transactions)
new_transactions = binarize(new_transactions)

In [2]:
def read_data(input_file):
    df=pd.read_csv(input_file)
    df['first_active_month']=pd.to_datetime(df['first_active_month'])
    df['elapsed_time']=(datetime.date(2018,2,1)-df['first_active_month'].dt.date).dt.days
    return df
train = read_data('Elo_merchant_recommendation/train.csv')
test=read_data('Elo_merchant_recommendation/test.csv')

target = train['target']
del train['target']

train.describe()
test.describe()
train.head()


,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time
0,2017-06-01,C_ID_92a2005557,5,2,1,245
1,2017-01-01,C_ID_3d0044924f,4,1,0,396
2,2016-08-01,C_ID_d639edf6cd,2,2,0,549
3,2017-09-01,C_ID_186d6a6901,4,3,0,153
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,92


In [3]:
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time
0,2017-04-01,C_ID_0ab67a22ab,3,3,1,306.0
1,2017-01-01,C_ID_130fd0cbdd,2,3,0,396.0
2,2017-08-01,C_ID_b709037bc5,5,1,1,184.0
3,2017-12-01,C_ID_d27d835a9f,2,1,0,62.0
4,2015-12-01,C_ID_2b5e3df5c2,5,1,1,793.0


In [4]:
new_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,1,C_ID_415bb3a509,107,0,1,B,307,M_ID_b0c793002c,1.0,-0.557574,2018-03-11 14:57:36,1.0,9.0,19.0
1,1,C_ID_415bb3a509,140,0,1,B,307,M_ID_88920c89e8,1.0,-0.569580,2018-03-19 18:53:37,1.0,9.0,19.0
2,1,C_ID_415bb3a509,330,0,1,B,507,M_ID_ad5237ef6b,2.0,-0.551037,2018-04-26 14:08:44,1.0,9.0,14.0
3,1,C_ID_415bb3a509,-1,1,1,B,661,M_ID_9e84cda3b1,1.0,-0.671925,2018-03-07 09:43:21,NaN,-1.0,8.0
4,1,C_ID_ef55cf8d4b,-1,1,1,B,166,M_ID_3c86fa3831,1.0,-0.659904,2018-03-22 21:07:53,NaN,-1.0,29.0


In [5]:
historical_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,1,C_ID_4e6213e9bc,88,0.0,0.0,A,80.0,M_ID_e020e9b302,-8.0,-0.703331,2017-06-25 15:33:07,1.0,16.0,37.0
1,1,C_ID_4e6213e9bc,88,0.0,0.0,A,367.0,M_ID_86ec983688,-7.0,-0.733128,2017-07-15 12:10:45,1.0,16.0,16.0
2,1,C_ID_4e6213e9bc,88,0.0,0.0,A,80.0,M_ID_979ed661fc,-6.0,-0.720386,2017-08-09 22:04:29,1.0,16.0,37.0
3,1,C_ID_4e6213e9bc,88,0.0,0.0,A,560.0,M_ID_e6d5ae8ea6,-5.0,-0.735352,2017-09-02 10:06:26,1.0,16.0,34.0
4,1,C_ID_4e6213e9bc,88,0.0,0.0,A,80.0,M_ID_e020e9b302,-11.0,-0.722865,2017-03-10 01:14:19,1.0,16.0,37.0


In [6]:

train.describe()


,feature_1,feature_2,feature_3,elapsed_time
count,201917.000000,201917.000000,201917.000000,201917.000000
mean,3.105311,1.745410,0.565569,381.978981
std,1.186160,0.751362,0.495683,293.710176
min,1.000000,1.000000,0.000000,0.000000
25%,2.000000,1.000000,0.000000,153.000000
50%,3.000000,2.000000,1.000000,306.000000
75%,4.000000,2.000000,1.000000,488.000000
max,5.000000,3.000000,1.000000,2284.000000


In [7]:
test.describe()

,feature_1,feature_2,feature_3,elapsed_time
count,123623.00000,123623.000000,123623.000000,123622.000000
mean,3.10926,1.741796,0.564377,381.117156
std,1.18911,0.749195,0.495840,292.558891
min,1.00000,1.000000,0.000000,31.000000
25%,2.00000,1.000000,0.000000,153.000000
50%,3.00000,2.000000,1.000000,306.000000
75%,4.00000,2.000000,1.000000,488.000000
max,5.00000,3.000000,1.000000,2284.000000


In [8]:
train.columns

Index(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'elapsed_time'],
      dtype='object')

In [9]:
test.columns

Index(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'elapsed_time'],
      dtype='object')

In [10]:
train.columns

Index(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'elapsed_time'],
      dtype='object')

In [11]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time
0,2017-06-01,C_ID_92a2005557,5,2,1,245
1,2017-01-01,C_ID_3d0044924f,4,1,0,396
2,2016-08-01,C_ID_d639edf6cd,2,2,0,549
3,2017-09-01,C_ID_186d6a6901,4,3,0,153
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,92


In [12]:
train.shape

(201917, 6)

In [13]:
test.shape

(123623, 6)

In [14]:
historical_transactions.shape


(1284319, 14)

In [15]:
new_transactions.shape

(1286538, 14)

In [16]:
sample_submission = pd.read_csv('Elo_merchant_recommendation/sample_submission.csv')
sample_submission.head()



,card_id,target
0,C_ID_0ab67a22ab,0
1,C_ID_130fd0cbdd,0
2,C_ID_b709037bc5,0
3,C_ID_d27d835a9f,0
4,C_ID_2b5e3df5c2,0


In [17]:
merchants = pd.read_csv('Elo_merchant_recommendation/merchants.csv')
merchants.head()

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,9.666667,3,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,1.750000,3,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,N,E,E,-82.13,260.000000,2,-82.13,260.000000,2,-82.13,260.000000,2,N,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,Y,E,E,NaN,1.666667,3,NaN,4.666667,6,NaN,3.833333,12,Y,-1,-1,NaN
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,Y,E,E,NaN,0.500000,3,NaN,0.361111,6,NaN,0.347222,12,Y,-1,-1,NaN


In [18]:
sample_submission.shape

(123623, 2)

In [19]:
test.shape

(123623, 6)

In [20]:
merchants.columns

Index(['merchant_id', 'merchant_group_id', 'merchant_category_id',
       'subsector_id', 'numerical_1', 'numerical_2', 'category_1',
       'most_recent_sales_range', 'most_recent_purchases_range',
       'avg_sales_lag3', 'avg_purchases_lag3', 'active_months_lag3',
       'avg_sales_lag6', 'avg_purchases_lag6', 'active_months_lag6',
       'avg_sales_lag12', 'avg_purchases_lag12', 'active_months_lag12',
       'category_4', 'city_id', 'state_id', 'category_2'],
      dtype='object')

In [21]:
merchants.columns

Index(['merchant_id', 'merchant_group_id', 'merchant_category_id',
       'subsector_id', 'numerical_1', 'numerical_2', 'category_1',
       'most_recent_sales_range', 'most_recent_purchases_range',
       'avg_sales_lag3', 'avg_purchases_lag3', 'active_months_lag3',
       'avg_sales_lag6', 'avg_purchases_lag6', 'active_months_lag6',
       'avg_sales_lag12', 'avg_purchases_lag12', 'active_months_lag12',
       'category_4', 'city_id', 'state_id', 'category_2'],
      dtype='object')

In [22]:

historical_transactions['month_diff']=((datetime.datetime.today()-historical_transactions['purchase_date']).dt.days)
historical_transactions['month_diff']+=historical_transactions['month_lag']
new_transactions['month_diff'] = ((datetime.datetime.today() - new_transactions['purchase_date']).dt.days)//30
new_transactions['month_diff'] += new_transactions['month_lag']

In [23]:
historical_transactions[:5]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,month_diff
0,1,C_ID_4e6213e9bc,88,0.0,0.0,A,80.0,M_ID_e020e9b302,-8.0,-0.703331,2017-06-25 15:33:07,1.0,16.0,37.0,568.0
1,1,C_ID_4e6213e9bc,88,0.0,0.0,A,367.0,M_ID_86ec983688,-7.0,-0.733128,2017-07-15 12:10:45,1.0,16.0,16.0,549.0
2,1,C_ID_4e6213e9bc,88,0.0,0.0,A,80.0,M_ID_979ed661fc,-6.0,-0.720386,2017-08-09 22:04:29,1.0,16.0,37.0,524.0
3,1,C_ID_4e6213e9bc,88,0.0,0.0,A,560.0,M_ID_e6d5ae8ea6,-5.0,-0.735352,2017-09-02 10:06:26,1.0,16.0,34.0,502.0
4,1,C_ID_4e6213e9bc,88,0.0,0.0,A,80.0,M_ID_e020e9b302,-11.0,-0.722865,2017-03-10 01:14:19,1.0,16.0,37.0,672.0


In [24]:
import sys
#import lightgbm as lgb

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

historical_transactions = pd.get_dummies(historical_transactions, columns=['category_1','category_3'])
new_transactions=pd.get_dummies(new_transactions, columns=['category_1','category_3'])
historical_transactions = reduce_mem_usage(historical_transactions)
new_transactions = reduce_mem_usage(new_transactions)

agg_fun = {'authorized_flag':['mean']}
auth_mean=historical_transactions.groupby(['card_id']).agg(agg_fun)
auth_mean.columns = ['_'.join(col).strip() for col in auth_mean.columns.values]
auth_mean.reset_index(inplace = True)

authorized_transactions = historical_transactions[historical_transactions['authorized_flag']==1]
historical_transactions = historical_transactions[historical_transactions['authorized_flag']==0]


Mem. usage decreased to 61.24 Mb (54.1% reduction)
Mem. usage decreased to 58.89 Mb (56.0% reduction)


In [25]:
historical_transactions.head()

,authorized_flag,card_id,city_id,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,month_diff,category_1_0.0,category_1_1.0,category_3_A,category_3_B,category_3_C
115,0,C_ID_4e6213e9bc,88,0.0,842.0,M_ID_22c9cfa265,-10.0,-0.730379,2017-04-07 12:58:09,1.0,16.0,37.0,645.0,1,0,1,0,0
132,0,C_ID_4e6213e9bc,88,0.0,367.0,M_ID_86ec983688,-5.0,-0.723782,2017-09-17 22:40:27,1.0,16.0,16.0,486.0,1,0,1,0,0
148,0,C_ID_4e6213e9bc,88,0.0,367.0,M_ID_86ec983688,-5.0,-0.723782,2017-09-17 22:40:26,1.0,16.0,16.0,486.0,1,0,1,0,0
168,0,C_ID_4e6213e9bc,333,0.0,605.0,M_ID_c2ae34c2ef,0.0,-0.664262,2018-02-20 10:57:50,1.0,9.0,2.0,336.0,1,0,1,0,0
213,0,C_ID_4e6213e9bc,88,0.0,560.0,M_ID_e6d5ae8ea6,-7.0,-0.738132,2017-07-08 07:33:31,1.0,16.0,34.0,556.0,1,0,1,0,0


In [26]:
historical_transactions.shape

(88772, 18)

In [27]:
#authorized_transactions = historical_transactions[historical_transactions['authorized_flag']==1]
authorized_transactions.shape

(1195547, 18)

In [28]:
historical_transactions.shape

(88772, 18)

In [29]:
#historical_transactions = historical_transactions[historical_transactions['authorized_flag']==0]
historical_transactions.shape

(88772, 18)

In [30]:
historical_transactions['purchase_month']=historical_transactions['purchase_date'].dt.month
authorized_transactions['purchase_month']=historical_transactions['purchase_date'].dt.month
new_transactions['purchase_month']=historical_transactions['purchase_date'].dt.month

In [31]:
historical_transactions.head()


,authorized_flag,card_id,city_id,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,month_diff,category_1_0.0,category_1_1.0,category_3_A,category_3_B,category_3_C,purchase_month
115,0,C_ID_4e6213e9bc,88,0.0,842.0,M_ID_22c9cfa265,-10.0,-0.730379,2017-04-07 12:58:09,1.0,16.0,37.0,645.0,1,0,1,0,0,4
132,0,C_ID_4e6213e9bc,88,0.0,367.0,M_ID_86ec983688,-5.0,-0.723782,2017-09-17 22:40:27,1.0,16.0,16.0,486.0,1,0,1,0,0,9
148,0,C_ID_4e6213e9bc,88,0.0,367.0,M_ID_86ec983688,-5.0,-0.723782,2017-09-17 22:40:26,1.0,16.0,16.0,486.0,1,0,1,0,0,9
168,0,C_ID_4e6213e9bc,333,0.0,605.0,M_ID_c2ae34c2ef,0.0,-0.664262,2018-02-20 10:57:50,1.0,9.0,2.0,336.0,1,0,1,0,0,2
213,0,C_ID_4e6213e9bc,88,0.0,560.0,M_ID_e6d5ae8ea6,-7.0,-0.738132,2017-07-08 07:33:31,1.0,16.0,34.0,556.0,1,0,1,0,0,7


In [32]:
new_transactions.head()


,authorized_flag,card_id,city_id,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,month_diff,category_1_0,category_1_1,category_3_A,category_3_B,category_3_C,purchase_month
0,1,C_ID_415bb3a509,107,1,307,M_ID_b0c793002c,1.0,-0.557617,2018-03-11 14:57:36,1.0,9.0,19.0,11.0,1,0,0,1,0,NaN
1,1,C_ID_415bb3a509,140,1,307,M_ID_88920c89e8,1.0,-0.569336,2018-03-19 18:53:37,1.0,9.0,19.0,11.0,1,0,0,1,0,NaN
2,1,C_ID_415bb3a509,330,1,507,M_ID_ad5237ef6b,2.0,-0.551270,2018-04-26 14:08:44,1.0,9.0,14.0,11.0,1,0,0,1,0,NaN
3,1,C_ID_415bb3a509,-1,1,661,M_ID_9e84cda3b1,1.0,-0.671875,2018-03-07 09:43:21,NaN,-1.0,8.0,11.0,0,1,0,1,0,NaN
4,1,C_ID_ef55cf8d4b,-1,1,166,M_ID_3c86fa3831,1.0,-0.659668,2018-03-22 21:07:53,NaN,-1.0,29.0,11.0,0,1,0,1,0,NaN


In [33]:
authorized_transactions.head()

,authorized_flag,card_id,city_id,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,month_diff,category_1_0.0,category_1_1.0,category_3_A,category_3_B,category_3_C,purchase_month
0,1,C_ID_4e6213e9bc,88,0.0,80.0,M_ID_e020e9b302,-8.0,-0.703331,2017-06-25 15:33:07,1.0,16.0,37.0,568.0,1,0,1,0,0,NaN
1,1,C_ID_4e6213e9bc,88,0.0,367.0,M_ID_86ec983688,-7.0,-0.733128,2017-07-15 12:10:45,1.0,16.0,16.0,549.0,1,0,1,0,0,NaN
2,1,C_ID_4e6213e9bc,88,0.0,80.0,M_ID_979ed661fc,-6.0,-0.720386,2017-08-09 22:04:29,1.0,16.0,37.0,524.0,1,0,1,0,0,NaN
3,1,C_ID_4e6213e9bc,88,0.0,560.0,M_ID_e6d5ae8ea6,-5.0,-0.735352,2017-09-02 10:06:26,1.0,16.0,34.0,502.0,1,0,1,0,0,NaN
4,1,C_ID_4e6213e9bc,88,0.0,80.0,M_ID_e020e9b302,-11.0,-0.722865,2017-03-10 01:14:19,1.0,16.0,37.0,672.0,1,0,1,0,0,NaN


In [34]:
def aggregate_transactions(history):
    history.loc[:,'purchase_date']=pd.DatetimeIndex(history['purchase_date']).astype(np.int64)*1e-9
    agg_func={
        'category_2':['sum','min'],
        'category_1_0.0': ['mean'],
        'category_1_1.0': ['mean'],
      #  'category_1_3.0': ['mean'],
      # 'category_2_4.0': ['mean'],
      #  'category_2_5.0': ['mean'],
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        'merchant_id':['nunique'],
        'merchant_category_id':['nunique'],
        'state_id':['nunique'],
        'city_id':['nunique'],
        'subsector_id':['nunique'],
        'purchase_amount':['sum','mean','max','min','std'],
        'installments':['sum','mean','max','min','std'],
        'purchase_month':['mean','max','min','std'],
        'purchase_date':[np.ptp,'min','max'],
        'month_lag':['mean','max','min','std'],
        'month_diff':['mean']
    }
    
    agg_history=history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    df = (history.groupby('card_id')
         .size()
         .reset_index(name='transactions_count'))
    
    agg_history=pd.merge(df, agg_history, on ='card_id', how='left')
    
    return agg_history
    

In [35]:
history = aggregate_transactions(historical_transactions)
history.columns = ['hist_' + c if c != 'card_id' else c for c in history.columns]
history[:5]


,card_id,hist_transactions_count,hist_category_2_sum,hist_category_2_min,hist_category_1_0.0_mean,hist_category_1_1.0_mean,hist_category_3_A_mean,hist_category_3_B_mean,hist_category_3_C_mean,hist_merchant_id_nunique,hist_merchant_category_id_nunique,hist_state_id_nunique,hist_city_id_nunique,hist_subsector_id_nunique,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_sum,hist_installments_mean,hist_installments_max,hist_installments_min,hist_installments_std,hist_purchase_month_mean,hist_purchase_month_max,hist_purchase_month_min,hist_purchase_month_std,hist_purchase_date_ptp,hist_purchase_date_min,hist_purchase_date_max,hist_month_lag_mean,hist_month_lag_max,hist_month_lag_min,hist_month_lag_std,hist_month_diff_mean
0,C_ID_00127204f5,17,30.0,1.0,1.000000,0.000000,0.882353,0.117647,0.000000,9,4,3,4,4,-11.129987,-0.654705,-0.544591,-0.737892,0.077477,2.0,0.117676,1.0,0.0,0.332031,6.235294,11,1,2.586731,26330201.0,1.488568e+09,1.514898e+09,-6.058594,0.0,-10.0,2.703125,550.50
1,C_ID_002198cdf1,4,4.0,1.0,1.000000,0.000000,1.000000,0.000000,0.000000,3,2,1,2,2,-1.986565,-0.496641,-0.296112,-0.711595,0.231850,0.0,0.000000,0.0,0.0,0.000000,1.500000,2,1,0.577350,1649891.0,1.516384e+09,1.518034e+09,-0.500000,0.0,-1.0,0.577148,358.50
2,C_ID_002780b4cf,8,31.0,1.0,0.875000,0.125000,0.125000,0.000000,0.875000,3,3,3,3,3,24.586979,3.073372,6.228790,-0.738538,2.922924,62.0,7.750000,10.0,0.0,4.199219,8.750000,12,8,1.488048,9963636.0,1.503746e+09,1.513710e+09,-3.250000,0.0,-4.0,1.488281,490.00
3,C_ID_002fc2a163,12,1.0,1.0,0.083333,0.916667,0.000000,0.916667,0.083333,5,5,2,2,4,-3.949704,-0.329142,2.498819,-0.716855,0.909067,15.0,1.250000,4.0,1.0,0.866211,3.916667,7,1,2.020726,15106045.0,1.483802e+09,1.498908e+09,-7.082031,-4.0,-10.0,2.019531,642.50
4,C_ID_0032aebb26,28,67.0,1.0,0.678571,0.321429,0.607143,0.357143,0.035714,16,10,3,3,7,-19.827137,-0.708112,-0.454342,-0.745405,0.067685,13.0,0.464355,3.0,0.0,0.692871,6.892857,11,1,3.881001,19591474.0,1.498875e+09,1.518467e+09,-3.677734,0.0,-7.0,2.326172,450.25


In [36]:
authorized = aggregate_transactions(authorized_transactions)
authorized.columns = ['auth_' + c if c != 'card_id' else c for c in authorized.columns]
authorized[:5]

,card_id,auth_transactions_count,auth_category_2_sum,auth_category_2_min,auth_category_1_0.0_mean,auth_category_1_1.0_mean,auth_category_3_A_mean,auth_category_3_B_mean,auth_category_3_C_mean,auth_merchant_id_nunique,auth_merchant_category_id_nunique,auth_state_id_nunique,auth_city_id_nunique,auth_subsector_id_nunique,auth_purchase_amount_sum,auth_purchase_amount_mean,auth_purchase_amount_max,auth_purchase_amount_min,auth_purchase_amount_std,auth_installments_sum,auth_installments_mean,auth_installments_max,auth_installments_min,auth_installments_std,auth_purchase_month_mean,auth_purchase_month_max,auth_purchase_month_min,auth_purchase_month_std,auth_purchase_date_ptp,auth_purchase_date_min,auth_purchase_date_max,auth_month_lag_mean,auth_month_lag_max,auth_month_lag_min,auth_month_lag_std,auth_month_diff_mean
0,C_ID_00127204f5,206,242.0,1.0,1.000000,0.000000,1.0,0.000000,0.000000,76,37,5,11,21,-136.686005,-0.663524,0.238230,-0.746878,0.095106,0.0,0.000000,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,28832026.0,1.488568e+09,1.517400e+09,-4.878906,0.0,-10.0,3.316406,515.5
1,C_ID_002198cdf1,91,91.0,1.0,1.000000,0.000000,1.0,0.000000,0.000000,56,28,1,3,15,-59.371716,-0.652436,-0.145847,-0.738643,0.105464,0.0,0.000000,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,15218051.0,1.502986e+09,1.518204e+09,-2.351562,0.0,-6.0,1.668945,412.0
2,C_ID_002780b4cf,46,106.0,1.0,0.478261,0.521739,0.0,0.913043,0.065217,18,14,4,6,10,-29.609726,-0.643690,1.472508,-0.740897,0.329161,55.0,1.195312,10.0,-1.0,1.375977,NaN,NaN,NaN,NaN,11251362.0,1.503329e+09,1.514580e+09,-1.500000,0.0,-4.0,1.394531,445.5
3,C_ID_002fc2a163,45,13.0,1.0,0.288889,0.711111,0.0,0.866667,0.133333,18,13,3,6,11,-27.819363,-0.618208,-0.190326,-0.745405,0.120500,53.0,1.177734,3.0,1.0,0.490234,NaN,NaN,NaN,NaN,24885672.0,1.483488e+09,1.508374e+09,-7.976562,-1.0,-10.0,1.839844,671.0
4,C_ID_0032aebb26,325,1135.0,1.0,1.000000,0.000000,1.0,0.000000,0.000000,78,26,3,5,13,-232.679642,-0.715937,-0.498339,-0.744654,0.031063,0.0,0.000000,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,20855318.0,1.498875e+09,1.519730e+09,-3.406250,0.0,-7.0,2.289062,442.5


In [37]:
new_transactions = new_transactions.rename(columns={'category_1_0':'category_1_0.0'})
new_transactions = new_transactions.rename(columns={'category_1_1':'category_1_1.0'})
new = aggregate_transactions(new_transactions)
new.columns = ['new_' + c if c != 'card_id' else c for c in new.columns]
new[:5]

,card_id,new_transactions_count,new_category_2_sum,new_category_2_min,new_category_1_0.0_mean,new_category_1_1.0_mean,new_category_3_A_mean,new_category_3_B_mean,new_category_3_C_mean,new_merchant_id_nunique,new_merchant_category_id_nunique,new_state_id_nunique,new_city_id_nunique,new_subsector_id_nunique,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_std,new_installments_sum,new_installments_mean,new_installments_max,new_installments_min,new_installments_std,new_purchase_month_mean,new_purchase_month_max,new_purchase_month_min,new_purchase_month_std,new_purchase_date_ptp,new_purchase_date_min,new_purchase_date_max,new_month_lag_mean,new_month_lag_max,new_month_lag_min,new_month_lag_std,new_month_diff_mean
0,C_ID_0001238066,26,35.0,1.0,0.923077,0.076923,0.0,0.807692,0.153846,25,15,4,8,9,-14.851562,-0.571289,-0.078308,-0.740723,0.173462,42,1.615385,10,-1,2.079941,2.0,2.0,2.0,NaN,5195343.0,1.519923e+09,1.525118e+09,1.345703,2.0,1.0,0.485107,10.804688
1,C_ID_0001793786,31,51.0,1.0,1.000000,0.000000,1.0,0.000000,0.000000,31,21,5,7,14,-0.229004,-0.007385,3.130859,-0.737793,0.947266,0,0.000000,0,0,0.000000,NaN,NaN,NaN,NaN,3981096.0,1.510761e+09,1.514742e+09,1.322266,2.0,1.0,0.475098,14.804688
2,C_ID_000183fdda,11,33.0,3.0,1.000000,0.000000,0.0,0.545455,0.363636,11,9,2,2,6,-6.589844,-0.599121,-0.107666,-0.732422,0.182861,16,1.454545,4,-1,1.293340,NaN,NaN,NaN,NaN,5106807.0,1.519994e+09,1.525100e+09,1.272461,2.0,1.0,0.467041,10.906250
3,C_ID_00032df08f,8,8.0,1.0,1.000000,0.000000,0.0,1.000000,0.000000,7,7,1,1,6,-2.791016,-0.348877,0.449219,-0.640625,0.366455,8,1.000000,1,1,0.000000,NaN,NaN,NaN,NaN,2059157.0,1.515165e+09,1.517224e+09,2.000000,2.0,2.0,0.000000,13.875000
4,C_ID_0003be3c83,6,6.0,1.0,1.000000,0.000000,0.0,0.666667,0.333333,6,6,1,2,5,-3.441406,-0.573730,-0.167969,-0.734863,0.234253,17,2.833333,10,1,3.600926,11.0,11.0,11.0,NaN,3476491.0,1.520693e+09,1.524170e+09,1.666992,2.0,1.0,0.516113,11.000000


In [38]:
new_transactions.head()

,authorized_flag,card_id,city_id,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,month_diff,category_1_0.0,category_1_1.0,category_3_A,category_3_B,category_3_C,purchase_month
0,1,C_ID_415bb3a509,107,1,307,M_ID_b0c793002c,1.0,-0.557617,1.520780e+09,1.0,9.0,19.0,11.0,1,0,0,1,0,NaN
1,1,C_ID_415bb3a509,140,1,307,M_ID_88920c89e8,1.0,-0.569336,1.521486e+09,1.0,9.0,19.0,11.0,1,0,0,1,0,NaN
2,1,C_ID_415bb3a509,330,1,507,M_ID_ad5237ef6b,2.0,-0.551270,1.524752e+09,1.0,9.0,14.0,11.0,1,0,0,1,0,NaN
3,1,C_ID_415bb3a509,-1,1,661,M_ID_9e84cda3b1,1.0,-0.671875,1.520416e+09,NaN,-1.0,8.0,11.0,0,1,0,1,0,NaN
4,1,C_ID_ef55cf8d4b,-1,1,166,M_ID_3c86fa3831,1.0,-0.659668,1.521753e+09,NaN,-1.0,29.0,11.0,0,1,0,1,0,NaN


In [39]:
new_transactions.rename(columns={'category_1_0':'category_1_0.0'})

,authorized_flag,card_id,city_id,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,month_diff,category_1_0.0,category_1_1.0,category_3_A,category_3_B,category_3_C,purchase_month
0,1,C_ID_415bb3a509,107,1,307,M_ID_b0c793002c,1.0,-0.557617,1.520780e+09,1.0,9.0,19.0,11.0,1,0,0,1,0,NaN
1,1,C_ID_415bb3a509,140,1,307,M_ID_88920c89e8,1.0,-0.569336,1.521486e+09,1.0,9.0,19.0,11.0,1,0,0,1,0,NaN
2,1,C_ID_415bb3a509,330,1,507,M_ID_ad5237ef6b,2.0,-0.551270,1.524752e+09,1.0,9.0,14.0,11.0,1,0,0,1,0,NaN
3,1,C_ID_415bb3a509,-1,1,661,M_ID_9e84cda3b1,1.0,-0.671875,1.520416e+09,NaN,-1.0,8.0,11.0,0,1,0,1,0,NaN
4,1,C_ID_ef55cf8d4b,-1,1,166,M_ID_3c86fa3831,1.0,-0.659668,1.521753e+09,NaN,-1.0,29.0,11.0,0,1,0,1,0,NaN
5,1,C_ID_ef55cf8d4b,231,1,367,M_ID_8874615e00,2.0,-0.632812,1.522674e+09,1.0,9.0,16.0,11.0,1,0,0,1,0,NaN
6,1,C_ID_ef55cf8d4b,69,1,333,M_ID_6d061b5ddc,1.0,5.265625,1.522267e+09,1.0,9.0,10.0,11.0,1,0,0,1,0,NaN
7,1,C_ID_ef55cf8d4b,231,1,307,M_ID_df1e022f41,2.0,-0.553711,1.522916e+09,1.0,9.0,19.0,11.0,1,0,0,1,0,NaN
8,1,C_ID_ef55cf8d4b,69,1,278,M_ID_d15eae0468,2.0,-0.596680,1.523126e+09,1.0,9.0,37.0,11.0,1,0,0,1,0,NaN
9,1,C_ID_ef55cf8d4b,69,1,437,M_ID_5f9bffd028,1.0,-0.607422,1.521310e+09,1.0,9.0,15.0,11.0,1,0,0,1,0,NaN


In [40]:
new = aggregate_transactions(new_transactions)
new.columns = ['new_' + c if c != 'card_id' else c for c in new.columns]
new[:5]

,card_id,new_transactions_count,new_category_2_sum,new_category_2_min,new_category_1_0.0_mean,new_category_1_1.0_mean,new_category_3_A_mean,new_category_3_B_mean,new_category_3_C_mean,new_merchant_id_nunique,new_merchant_category_id_nunique,new_state_id_nunique,new_city_id_nunique,new_subsector_id_nunique,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_std,new_installments_sum,new_installments_mean,new_installments_max,new_installments_min,new_installments_std,new_purchase_month_mean,new_purchase_month_max,new_purchase_month_min,new_purchase_month_std,new_purchase_date_ptp,new_purchase_date_min,new_purchase_date_max,new_month_lag_mean,new_month_lag_max,new_month_lag_min,new_month_lag_std,new_month_diff_mean
0,C_ID_0001238066,26,35.0,1.0,0.923077,0.076923,0.0,0.807692,0.153846,25,15,4,8,9,-14.851562,-0.571289,-0.078308,-0.740723,0.173462,42,1.615385,10,-1,2.079941,2.0,2.0,2.0,NaN,0.005195,1.519923,1.525118,1.345703,2.0,1.0,0.485107,10.804688
1,C_ID_0001793786,31,51.0,1.0,1.000000,0.000000,1.0,0.000000,0.000000,31,21,5,7,14,-0.229004,-0.007385,3.130859,-0.737793,0.947266,0,0.000000,0,0,0.000000,NaN,NaN,NaN,NaN,0.003981,1.510761,1.514742,1.322266,2.0,1.0,0.475098,14.804688
2,C_ID_000183fdda,11,33.0,3.0,1.000000,0.000000,0.0,0.545455,0.363636,11,9,2,2,6,-6.589844,-0.599121,-0.107666,-0.732422,0.182861,16,1.454545,4,-1,1.293340,NaN,NaN,NaN,NaN,0.005107,1.519994,1.525100,1.272461,2.0,1.0,0.467041,10.906250
3,C_ID_00032df08f,8,8.0,1.0,1.000000,0.000000,0.0,1.000000,0.000000,7,7,1,1,6,-2.791016,-0.348877,0.449219,-0.640625,0.366455,8,1.000000,1,1,0.000000,NaN,NaN,NaN,NaN,0.002059,1.515165,1.517224,2.000000,2.0,2.0,0.000000,13.875000
4,C_ID_0003be3c83,6,6.0,1.0,1.000000,0.000000,0.0,0.666667,0.333333,6,6,1,2,5,-3.441406,-0.573730,-0.167969,-0.734863,0.234253,17,2.833333,10,1,3.600926,11.0,11.0,11.0,NaN,0.003476,1.520693,1.524170,1.666992,2.0,1.0,0.516113,11.000000


In [41]:
new_transactions.columns

Index(['authorized_flag', 'card_id', 'city_id', 'installments',
       'merchant_category_id', 'merchant_id', 'month_lag', 'purchase_amount',
       'purchase_date', 'category_2', 'state_id', 'subsector_id', 'month_diff',
       'category_1_0.0', 'category_1_1.0', 'category_3_A', 'category_3_B',
       'category_3_C', 'purchase_month'],
      dtype='object')

In [42]:
new_transactions.columns

Index(['authorized_flag', 'card_id', 'city_id', 'installments',
       'merchant_category_id', 'merchant_id', 'month_lag', 'purchase_amount',
       'purchase_date', 'category_2', 'state_id', 'subsector_id', 'month_diff',
       'category_1_0.0', 'category_1_1.0', 'category_3_A', 'category_3_B',
       'category_3_C', 'purchase_month'],
      dtype='object')

In [43]:
new = aggregate_transactions(new_transactions)
new.columns = ['new_' + c if c != 'card_id' else c for c in new.columns]
new[:5]

,card_id,new_transactions_count,new_category_2_sum,new_category_2_min,new_category_1_0.0_mean,new_category_1_1.0_mean,new_category_3_A_mean,new_category_3_B_mean,new_category_3_C_mean,new_merchant_id_nunique,new_merchant_category_id_nunique,new_state_id_nunique,new_city_id_nunique,new_subsector_id_nunique,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_std,new_installments_sum,new_installments_mean,new_installments_max,new_installments_min,new_installments_std,new_purchase_month_mean,new_purchase_month_max,new_purchase_month_min,new_purchase_month_std,new_purchase_date_ptp,new_purchase_date_min,new_purchase_date_max,new_month_lag_mean,new_month_lag_max,new_month_lag_min,new_month_lag_std,new_month_diff_mean
0,C_ID_0001238066,26,35.0,1.0,0.923077,0.076923,0.0,0.807692,0.153846,25,15,4,8,9,-14.851562,-0.571289,-0.078308,-0.740723,0.173462,42,1.615385,10,-1,2.079941,2.0,2.0,2.0,NaN,0.0,1.000000e-09,1.000000e-09,1.345703,2.0,1.0,0.485107,10.804688
1,C_ID_0001793786,31,51.0,1.0,1.000000,0.000000,1.0,0.000000,0.000000,31,21,5,7,14,-0.229004,-0.007385,3.130859,-0.737793,0.947266,0,0.000000,0,0,0.000000,NaN,NaN,NaN,NaN,0.0,1.000000e-09,1.000000e-09,1.322266,2.0,1.0,0.475098,14.804688
2,C_ID_000183fdda,11,33.0,3.0,1.000000,0.000000,0.0,0.545455,0.363636,11,9,2,2,6,-6.589844,-0.599121,-0.107666,-0.732422,0.182861,16,1.454545,4,-1,1.293340,NaN,NaN,NaN,NaN,0.0,1.000000e-09,1.000000e-09,1.272461,2.0,1.0,0.467041,10.906250
3,C_ID_00032df08f,8,8.0,1.0,1.000000,0.000000,0.0,1.000000,0.000000,7,7,1,1,6,-2.791016,-0.348877,0.449219,-0.640625,0.366455,8,1.000000,1,1,0.000000,NaN,NaN,NaN,NaN,0.0,1.000000e-09,1.000000e-09,2.000000,2.0,2.0,0.000000,13.875000
4,C_ID_0003be3c83,6,6.0,1.0,1.000000,0.000000,0.0,0.666667,0.333333,6,6,1,2,5,-3.441406,-0.573730,-0.167969,-0.734863,0.234253,17,2.833333,10,1,3.600926,11.0,11.0,11.0,NaN,0.0,1.000000e-09,1.000000e-09,1.666992,2.0,1.0,0.516113,11.000000


In [44]:
new = aggregate_transactions(new_transactions)
new.columns = ['new_' + c if c != 'card_id' else c for c in new.columns]
new[:5]

,card_id,new_transactions_count,new_category_2_sum,new_category_2_min,new_category_1_0.0_mean,new_category_1_1.0_mean,new_category_3_A_mean,new_category_3_B_mean,new_category_3_C_mean,new_merchant_id_nunique,new_merchant_category_id_nunique,new_state_id_nunique,new_city_id_nunique,new_subsector_id_nunique,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_std,new_installments_sum,new_installments_mean,new_installments_max,new_installments_min,new_installments_std,new_purchase_month_mean,new_purchase_month_max,new_purchase_month_min,new_purchase_month_std,new_purchase_date_ptp,new_purchase_date_min,new_purchase_date_max,new_month_lag_mean,new_month_lag_max,new_month_lag_min,new_month_lag_std,new_month_diff_mean
0,C_ID_0001238066,26,35.0,1.0,0.923077,0.076923,0.0,0.807692,0.153846,25,15,4,8,9,-14.851562,-0.571289,-0.078308,-0.740723,0.173462,42,1.615385,10,-1,2.079941,2.0,2.0,2.0,NaN,0.0,0.0,0.0,1.345703,2.0,1.0,0.485107,10.804688
1,C_ID_0001793786,31,51.0,1.0,1.000000,0.000000,1.0,0.000000,0.000000,31,21,5,7,14,-0.229004,-0.007385,3.130859,-0.737793,0.947266,0,0.000000,0,0,0.000000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.322266,2.0,1.0,0.475098,14.804688
2,C_ID_000183fdda,11,33.0,3.0,1.000000,0.000000,0.0,0.545455,0.363636,11,9,2,2,6,-6.589844,-0.599121,-0.107666,-0.732422,0.182861,16,1.454545,4,-1,1.293340,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.272461,2.0,1.0,0.467041,10.906250
3,C_ID_00032df08f,8,8.0,1.0,1.000000,0.000000,0.0,1.000000,0.000000,7,7,1,1,6,-2.791016,-0.348877,0.449219,-0.640625,0.366455,8,1.000000,1,1,0.000000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,2.000000,2.0,2.0,0.000000,13.875000
4,C_ID_0003be3c83,6,6.0,1.0,1.000000,0.000000,0.0,0.666667,0.333333,6,6,1,2,5,-3.441406,-0.573730,-0.167969,-0.734863,0.234253,17,2.833333,10,1,3.600926,11.0,11.0,11.0,NaN,0.0,0.0,0.0,1.666992,2.0,1.0,0.516113,11.000000


In [45]:
def aggregate_per_month(history):
    grouped = history.groupby(['card_id','month_lag'])
    
    agg_func={
        'purchase_amount':['count','sum','mean','min','max','std'],
        'installments':['count','sum','mean','min','max','std']
    }
    
    intermediate_group = grouped.agg(agg_func)
    intermediate_group.columns = ['_'.join(col).strip() for col in intermediate_group.columns.values]
    intermediate_group.reset_index(inplace=True)
    
    final_group = intermediate_group.groupby('card_id').agg(['mean', 'std'])
    final_group.columns = ['_'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace=True)
    
    return final_group

#-----------------------------------------------------------------------
final_group=aggregate_per_month(historical_transactions)
final_group[:10]
    

,card_id,month_lag_mean,month_lag_std,purchase_amount_count_mean,purchase_amount_count_std,purchase_amount_sum_mean,purchase_amount_sum_std,purchase_amount_mean_mean,purchase_amount_mean_std,purchase_amount_min_mean,purchase_amount_min_std,purchase_amount_max_mean,purchase_amount_max_std,purchase_amount_std_mean,purchase_amount_std_std,installments_count_mean,installments_count_std,installments_sum_mean,installments_sum_std,installments_mean_mean,installments_mean_std,installments_min_mean,installments_min_std,installments_max_mean,installments_max_std,installments_std_mean,installments_std_std
0,C_ID_00127204f5,-5.000000,3.559026,2.428571,2.299068,-1.589998,1.360800,-0.683448,0.048446,-0.708695,0.030214,-0.671378,0.064815,0.037385,0.042220,2.428571,2.299068,0.285645,0.487793,0.163208,0.372803,0.142822,0.377930,0.285645,0.487793,0.125977,0.218140
1,C_ID_002198cdf1,-0.500000,0.707107,2.000000,0.000000,-0.993282,0.567181,-0.496641,0.283591,-0.503854,0.293791,-0.489429,0.273390,0.010200,0.014425,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,C_ID_002780b4cf,-2.000000,2.000000,2.666667,2.886751,8.195660,15.428233,0.970487,2.913966,0.262523,1.687825,1.601716,4.007253,2.078258,NaN,2.666667,2.886751,20.671875,34.062500,4.000000,5.292969,4.000000,5.292969,4.000000,5.292969,0.000000,NaN
3,C_ID_002fc2a163,-7.000000,2.160247,1.714286,0.951190,-0.564243,1.480214,-0.140225,1.172430,-0.180273,1.189471,-0.077477,1.155895,0.144571,0.147750,1.714286,0.951190,2.142578,1.214844,1.428711,1.133789,1.428711,1.133789,1.428711,1.133789,0.000000,0.000000
4,C_ID_0032aebb26,-3.666667,2.804758,4.666667,2.581989,-3.304523,1.817188,-0.712886,0.018349,-0.738142,0.007201,-0.632666,0.122553,0.050049,0.048400,4.666667,2.581989,2.166016,2.136719,0.388916,0.330811,0.000000,0.000000,1.000000,1.095703,0.588867,0.419189
5,C_ID_0037be6c74,-7.428571,4.117327,1.857143,1.463850,-1.293128,1.001118,-0.702758,0.017121,-0.707120,0.015356,-0.694777,0.025889,0.016707,0.009413,1.857143,1.463850,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,C_ID_003839dd44,-6.714286,4.608481,4.142857,5.336309,-1.879660,4.432688,-0.446511,0.638319,-0.698346,0.048356,0.078000,2.014173,0.498743,1.094473,4.142857,5.336309,4.855469,6.742188,0.893066,1.153320,0.428467,0.534668,2.142578,4.375000,1.169922,2.617188
7,C_ID_0039c126cb,-5.000000,NaN,1.000000,NaN,-0.641196,NaN,-0.641196,NaN,-0.641196,NaN,-0.641196,NaN,NaN,NaN,1.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN
8,C_ID_0046db9f8a,-2.000000,1.000000,2.666667,1.154701,0.391806,2.104310,-0.031870,0.680210,-0.413069,0.256866,0.997745,2.297964,0.723055,1.073908,2.666667,1.154701,5.000000,8.890625,1.250000,2.384766,1.000000,2.000000,2.000000,3.605469,0.500000,0.866211
9,C_ID_004caeab1b,-4.800000,5.069517,3.400000,3.361547,-1.916160,2.418954,-0.384275,0.638060,-0.394936,0.643505,-0.375302,0.633818,0.017620,0.020427,3.400000,3.361547,0.199951,0.447266,0.199951,0.447266,0.199951,0.447266,0.199951,0.447266,0.000000,0.000000


In [46]:
history.columns

Index(['card_id', 'hist_transactions_count', 'hist_category_2_sum',
       'hist_category_2_min', 'hist_category_1_0.0_mean',
       'hist_category_1_1.0_mean', 'hist_category_3_A_mean',
       'hist_category_3_B_mean', 'hist_category_3_C_mean',
       'hist_merchant_id_nunique', 'hist_merchant_category_id_nunique',
       'hist_state_id_nunique', 'hist_city_id_nunique',
       'hist_subsector_id_nunique', 'hist_purchase_amount_sum',
       'hist_purchase_amount_mean', 'hist_purchase_amount_max',
       'hist_purchase_amount_min', 'hist_purchase_amount_std',
       'hist_installments_sum', 'hist_installments_mean',
       'hist_installments_max', 'hist_installments_min',
       'hist_installments_std', 'hist_purchase_month_mean',
       'hist_purchase_month_max', 'hist_purchase_month_min',
       'hist_purchase_month_std', 'hist_purchase_date_ptp',
       'hist_purchase_date_min', 'hist_purchase_date_max',
       'hist_month_lag_mean', 'hist_month_lag_max', 'hist_month_lag_min',
      

In [47]:
historical_transactions.columns

Index(['authorized_flag', 'card_id', 'city_id', 'installments',
       'merchant_category_id', 'merchant_id', 'month_lag', 'purchase_amount',
       'purchase_date', 'category_2', 'state_id', 'subsector_id', 'month_diff',
       'category_1_0.0', 'category_1_1.0', 'category_3_A', 'category_3_B',
       'category_3_C', 'purchase_month'],
      dtype='object')

In [48]:
new_transactions.columns

Index(['authorized_flag', 'card_id', 'city_id', 'installments',
       'merchant_category_id', 'merchant_id', 'month_lag', 'purchase_amount',
       'purchase_date', 'category_2', 'state_id', 'subsector_id', 'month_diff',
       'category_1_0.0', 'category_1_1.0', 'category_3_A', 'category_3_B',
       'category_3_C', 'purchase_month'],
      dtype='object')

In [49]:
def successive_aggregates(df, field1, field2):
    t=df.groupby(['card_id',field1])[field2].mean()
    u=pd.DataFrame(t).reset_index().groupby('card_id')[field2].agg(['mean','min','max','std'])
    u.columns = [field1 + '_' + field2 + '_' + col for col in u.columns.values]
    u.reset_index(inplace=True)
    return u
#---------------------------------------------------------------------------------------------------
additional_fields = successive_aggregates(new_transactions, 'category_2', 'purchase_amount')
additional_fields = additional_fields.merge(successive_aggregates(new_transactions, 'installments', 'purchase_amount'),
                                            on = 'card_id', how='left')
additional_fields = additional_fields.merge(successive_aggregates(new_transactions, 'city_id', 'purchase_amount'),
                                            on = 'card_id', how='left')
additional_fields = additional_fields.merge(successive_aggregates(new_transactions, 'category_2', 'installments'),
                                            on = 'card_id', how='left')

In [50]:
#training the model - merging all the dataframes
train = pd.merge(train,history,on='card_id',how='left')
test=pd.merge(test,history,on='card_id',how='left')

train = pd.merge(train, authorized, on='card_id', how='left')
test=pd.merge(test,authorized, on='card_id', how='left')

train = pd.merge(train, final_group, on='card_id', how='left')
test=pd.merge(test, final_group, on='card_id', how='left')

train = pd.merge(train, auth_mean, on ='card_id', how='left')
test=pd.merge(test, auth_mean, on='card_id', how='left')

train = pd.merge(train, additional_fields, on ='card_id', how='left')
test=pd.merge(test, additional_fields, on='card_id', how='left')


In [51]:
new.shape

(126550, 36)

In [52]:
final_group.shape

(5801, 27)

In [53]:
auth_mean.shape

(6135, 2)

In [54]:
additional_fields.shape

(124774, 17)

In [55]:
#cols=[c for c in train if c.startswith('hist')]
#train.loc[train['hist_transactions_count'].isnull(), cols]=0
#test.loc[test['hist_transactions_count'].isnull(), cols]=0

# cols = [c for c in train if c.startswith('new')]
# train.loc[train['new_transactions_count'].isnull(), cols] = 0
# test.loc[test['new_transactions_count'].isnull(), cols] = 0

# cols = [c for c in train if c.endswith('std')]
# for c in cols:
#     train.loc[train[c].isnull(), c] = 0
#     test.loc[test[c].isnull(), c] = 0

# train['transactions_ratio'] = train['new_transactions_count'] / train['hist_transactions_count']
# test['transactions_ratio'] = test['new_transactions_count'] / test['hist_transactions_count']

# hist_columns = [(c, c.replace('new', 'auth')) for c in train.columns if 'hist' in c]
# for c in hist_columns:
#     col_name = 'ratio_{}_{}'.format(c[0], c[1])
#     train[col_name] = train[c[0]] / train[c[1]]
#     test[col_name] = test[c[0]] / test[c[1]]

# unimportant_features = [
#     'auth_category_2_1.0_mean',
#     'auth_category_2_2.0_mean',
#     'auth_category_2_3.0_mean',
#     'auth_category_2_5.0_mean',
#     'hist_category_2_3.0_mean',
#     'hist_category_2_4.0_mean',
#     'hist_category_2_5.0_mean',
#     'hist_category_3_A_mean',
#     'hist_installments_min',
#     'hist_installments_std',
#     'hist_month_lag_std',
#     'hist_purchase_amount_max',
#     'hist_purchase_month_max',
#     'hist_purchase_month_min',
#     'hist_purchase_month_std',
#     'installments_min_mean',
#     'new_category_2_1.0_mean',
#     'new_category_2_2.0_mean',
#     'new_category_2_3.0_mean',
#     'new_category_2_5.0_mean',
#     'new_city_id_nunique',
#     'new_installments_std',
#     'new_state_id_nunique',
#     'purchase_amount_mean_mean'
# ]
features = [c for c in train.columns if c not in ['card_id', 'first_active_month']]
#features = [f for f in features if f not in unimportant_features]
categorical_feats = ['feature_2', 'feature_3']


In [56]:
param = {'num_leaves': 111,
         'min_data_in_leaf': 149, 
         'objective':'regression',
         'max_depth': 9,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.7522,
         "bagging_freq": 1,
         "bagging_fraction": 0.7083 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.2634,
         "random_state": 133,
         "verbosity": -1}


In [57]:
folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof=np.zeros(len(train))
predictions = np.zeros(len(test))

start = time.time()
feature_importance_df = pd.DataFrame()

for fold_,(trn_idx,val_idx) in enumerate(folds.split(train.values,target.values)):
    print("fold n{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                           label=target.iloc[trn_idx],
                           categorical_feature=categorical_feats
                          )
    val_data = lgb.Dataset(train.iloc[val_idx][features],
                           label=target.iloc[val_idx],
                           categorical_feature=categorical_feats
                          )

    num_round = 10000
    clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=100,
                    early_stopping_rounds = 200)
    
    oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(mean_squared_error(oof, target)**0.5))



fold n0


C:\Users\mc9131\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\mc9131\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.828	valid_1's rmse: 3.8736
[200]	training's rmse: 3.81956	valid_1's rmse: 3.86876
[300]	training's rmse: 3.81412	valid_1's rmse: 3.86654
[400]	training's rmse: 3.81059	valid_1's rmse: 3.86548
[500]	training's rmse: 3.80808	valid_1's rmse: 3.86501
[600]	training's rmse: 3.80619	valid_1's rmse: 3.86488
[700]	training's rmse: 3.80465	valid_1's rmse: 3.86479
[800]	training's rmse: 3.80351	valid_1's rmse: 3.86475
[900]	training's rmse: 3.8025	valid_1's rmse: 3.86474
[1000]	training's rmse: 3.80157	valid_1's rmse: 3.86483
Early stopping, best iteration is:
[819]	training's rmse: 3.80327	valid_1's rmse: 3.86473
fold n1


C:\Users\mc9131\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\mc9131\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.84748	valid_1's rmse: 3.79522
[200]	training's rmse: 3.83898	valid_1's rmse: 3.79038
[300]	training's rmse: 3.83377	valid_1's rmse: 3.78823
[400]	training's rmse: 3.83029	valid_1's rmse: 3.78735
[500]	training's rmse: 3.82783	valid_1's rmse: 3.78693
[600]	training's rmse: 3.82595	valid_1's rmse: 3.78673
[700]	training's rmse: 3.82448	valid_1's rmse: 3.78673
[800]	training's rmse: 3.8233	valid_1's rmse: 3.78691
Early stopping, best iteration is:
[693]	training's rmse: 3.82459	valid_1's rmse: 3.78672
fold n2


...

Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.85303	valid_1's rmse: 3.77098
[200]	training's rmse: 3.84426	valid_1's rmse: 3.7668
[300]	training's rmse: 3.83864	valid_1's rmse: 3.76511
[400]	training's rmse: 3.83504	valid_1's rmse: 3.76449
[500]	training's rmse: 3.83257	valid_1's rmse: 3.76435
[600]	training's rmse: 3.83059	valid_1's rmse: 3.76449
[700]	training's rmse: 3.829	valid_1's rmse: 3.76467
Early stopping, best iteration is:
[531]	training's rmse: 3.83189	valid_1's rmse: 3.76433
fold n3


C:\Users\mc9131\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\mc9131\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.80127	valid_1's rmse: 3.97679
[200]	training's rmse: 3.79288	valid_1's rmse: 3.97253
[300]	training's rmse: 3.78742	valid_1's rmse: 3.97055
[400]	training's rmse: 3.78389	valid_1's rmse: 3.96965
[500]	training's rmse: 3.78146	valid_1's rmse: 3.96932
[600]	training's rmse: 3.7796	valid_1's rmse: 3.96921
[700]	training's rmse: 3.77824	valid_1's rmse: 3.9692
[800]	training's rmse: 3.77715	valid_1's rmse: 3.96923
[900]	training's rmse: 3.77614	valid_1's rmse: 3.96926
Early stopping, best iteration is:
[722]	training's rmse: 3.77796	valid_1's rmse: 3.96917
fold n4


C:\Users\mc9131\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\mc9131\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.85003	valid_1's rmse: 3.78378
[200]	training's rmse: 3.84131	valid_1's rmse: 3.77948
[300]	training's rmse: 3.83588	valid_1's rmse: 3.77771
[400]	training's rmse: 3.83232	valid_1's rmse: 3.77703
[500]	training's rmse: 3.82988	valid_1's rmse: 3.77679
[600]	training's rmse: 3.82793	valid_1's rmse: 3.77678
[700]	training's rmse: 3.82638	valid_1's rmse: 3.77673
[800]	training's rmse: 3.82525	valid_1's rmse: 3.7768
Early stopping, best iteration is:
[691]	training's rmse: 3.82649	valid_1's rmse: 3.77672
CV score: 3.83311 
